# Read in packages

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt

# Read in Seattle data

In [ ]:
seattle_cal_df = pd.read_csv('data/boston-airbnb-open-data/calendar.csv',parse_dates=True)
seattle_listings_df = pd.read_csv('data/boston-airbnb-open-data/listings.csv',parse_dates=True)
seattle_reviews_df = pd.read_csv('data/boston-airbnb-open-data/reviews.csv')